In [82]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import StratifiedShuffleSplit
import tensorflow as tf

In [83]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout
from keras.optimizers import SGD
from keras.utils import np_utils

In [84]:
classes = ['Acer_Capillipes', 'Acer_Circinatum', 'Acer_Mono', 'Acer_Opalus', 'Acer_Palmatum', 'Acer_Pictum', 'Acer_Platanoids', 'Acer_Rubrum', 'Acer_Rufinerve', 'Acer_Saccharinum', 'Alnus_Cordata', 'Alnus_Maximowiczii', 'Alnus_Rubra', 'Alnus_Sieboldiana', 'Alnus_Viridis', 'Arundinaria_Simonii', 'Betula_Austrosinensis', 'Betula_Pendula', 'Callicarpa_Bodinieri', 'Castanea_Sativa', 'Celtis_Koraiensis', 'Cercis_Siliquastrum', 'Cornus_Chinensis', 'Cornus_Controversa', 'Cornus_Macrophylla', 'Cotinus_Coggygria', 'Crataegus_Monogyna', 'Cytisus_Battandieri', 'Eucalyptus_Glaucescens', 'Eucalyptus_Neglecta', 'Eucalyptus_Urnigera', 'Fagus_Sylvatica', 'Ginkgo_Biloba', 'Ilex_Aquifolium', 'Ilex_Cornuta', 'Liquidambar_Styraciflua', 'Liriodendron_Tulipifera', 'Lithocarpus_Cleistocarpus', 'Lithocarpus_Edulis', 'Magnolia_Heptapeta', 'Magnolia_Salicifolia', 'Morus_Nigra', 'Olea_Europaea', 'Phildelphus', 'Populus_Adenopoda', 'Populus_Grandidentata', 'Populus_Nigra', 'Prunus_Avium', 'Prunus_X_Shmittii', 'Pterocarya_Stenoptera', 'Quercus_Afares', 'Quercus_Agrifolia', 'Quercus_Alnifolia', 'Quercus_Brantii', 'Quercus_Canariensis', 'Quercus_Castaneifolia', 'Quercus_Cerris', 'Quercus_Chrysolepis', 'Quercus_Coccifera', 'Quercus_Coccinea', 'Quercus_Crassifolia', 'Quercus_Crassipes', 'Quercus_Dolicholepis', 'Quercus_Ellipsoidalis', 'Quercus_Greggii', 'Quercus_Hartwissiana', 'Quercus_Ilex', 'Quercus_Imbricaria', 'Quercus_Infectoria_sub', 'Quercus_Kewensis', 'Quercus_Nigra', 'Quercus_Palustris', 'Quercus_Phellos', 'Quercus_Phillyraeoides', 'Quercus_Pontica', 'Quercus_Pubescens', 'Quercus_Pyrenaica', 'Quercus_Rhysophylla', 'Quercus_Rubra', 'Quercus_Semecarpifolia', 'Quercus_Shumardii', 'Quercus_Suber', 'Quercus_Texana', 'Quercus_Trojana', 'Quercus_Variabilis', 'Quercus_Vulcanica', 'Quercus_x_Hispanica', 'Quercus_x_Turneri', 'Rhododendron_x_Russellianum', 'Salix_Fragilis', 'Salix_Intergra', 'Sorbus_Aria', 'Tilia_Oliveri', 'Tilia_Platyphyllos', 'Tilia_Tomentosa', 'Ulmus_Bergmanniana', 'Viburnum_Tinus', 'Viburnum_x_Rhytidophylloides', 'Zelkova_Serrata']
num_classes = len(classes)
num_labels = 990
print(len(classes))

99


In [85]:
train = pd.read_csv('train.csv')
parent_data = train.copy()
test = pd.read_csv('test.csv')
print(train.shape)

(990, 194)


In [86]:
def dense_to_one_hot(train_list):
    """Convert class labels from scalars to one-hot vectors."""
    labels_one_hot = np.zeros((len(train_list), num_classes))
    for i in xrange(len(train_list)):
    	labels_one_hot[i][train_list[i]] = 1 
    return labels_one_hot

In [87]:
train_id = train.pop('id')
train_labels = train.pop('species')
train_labels = LabelEncoder().fit(train_labels).transform(train_labels)
print train_labels.shape
train_labels_one_hot = dense_to_one_hot(train_labels)
print (train_labels_one_hot.shape)

(990,)
(990, 99)


In [88]:
train_data = StandardScaler().fit(train).transform(train)
print train_data.shape

(990, 192)


In [89]:
test = pd.read_csv('test.csv')
test_index = test.pop('id')
test_data = StandardScaler().fit(test).transform(test)
print test_data.shape

(594, 192)


In [90]:
sss = StratifiedShuffleSplit(train_labels,test_size=0.1, random_state=23)

for train_index, test_index in sss:
    train_data_2, valid_data = train_data[train_index], train_data[test_index]
    train_labels_2, valid_labels = train_labels[train_index], train_labels[test_index]
    train_labels_one_hot_2, valid_labels_one_hot = train_labels_one_hot[train_index], train_labels_one_hot[test_index]
print valid_data.shape, valid_labels_one_hot.shape, train_labels.shape

(99, 192) (99, 99) (990,)


In [91]:
nb_features = 64   
nb_class = len(classes)

train_cnn = np.zeros((len(train_data_2), nb_features, 3))
valid_cnn = np.zeros((len(valid_data), nb_features, 3))
test_cnn = np.zeros((len(test_data), nb_features, 3))

for in xrange(3):
    train_cnn[:, :, i] = train_data_2[:, i:(i+1)*nb_features]
    valid_cnn[:, :, i] = valid_data[:, i:(i+1)*nb_features]
    test_cnn[:, :, i] = test_data[:,  i:(i+1)*nb_features]
    

In [92]:
model = Sequential()
model.add(Convolution1D(nb_filter=512, filter_length=1, input_shape=(nb_features, 3)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(nb_class))
model.add(Activation('softmax'))


In [93]:
sgd = SGD(lr=0.01, nesterov=True, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])


In [94]:
train_labels_one_hot = np_utils.to_categorical(train_labels_2, nb_class)
test_labels_one_hot = np_utils.to_categorical(valid_labels, nb_class)


In [95]:
nb_epoch = 1
model.fit(train_cnn, train_labels_one_hot, nb_epoch=nb_epoch, validation_data=(valid_cnn, test_labels_one_hot), batch_size=16)

Train on 891 samples, validate on 99 samples
Epoch 1/1
891/891 [==============================] - 79s - loss: 4.3519 - acc: 0.1459 - val_loss: 3.3642 - val_acc: 0.5253


In [97]:

data = pd.read_csv('test.csv')

index = data.pop('id')

test_pred = model.predict_proba(test_cnn)
test_pred = pd.DataFrame(test_pred,index=index,columns=classes)
print test_pred.size
fp = open('submission_cnn_kernel.csv','w')
fp.write(test_pred.to_csv())

594/594 [==============================] - 3s     
58806
